# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [18]:
#include <stdio.h>

int main(int argc, char** argv)
{
    double S = 200;
    int N = 5;
    double J = 1;
    
    printf("Parcelas do financiamento: ");
    
    double parcela = S;
    
    for (int i = 0; i < N; i++)
    {
        printf("%.2f ", parcela);
        parcela *= 1 + J/100;
    }
}

Parcelas do financiamento: 200.00 202.00 204.02 206.06 208.12 

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [28]:
#include <stdio.h>

double proximaParcela(double parcela_atual, double taxa_juros)
{
    return parcela_atual * (1 + taxa_juros/100);
}

int main(int argc, char** argv)
{
    double S = 200;
    int N = 5;
    double J = 1;
    
    printf("Parcelas do financiamento: ");
    double parcela = S;
    
    for (int i = 0; i < N; i++)
    {
        printf("%.2f ", parcela);
        parcela = proximaParcela(parcela, J);
    }
}

Parcelas do financiamento: 200.00 202.00 204.02 206.06 208.12 

## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [35]:
#include <stdio.h>

double g_parcela_atual = 0;
int g_numero_parcelas = 0;
double g_taxa_juros = 0;

void inicializarParametros(double parcela_inicial, int numero_parcelas, double taxa_juros)
{
    g_parcela_atual = parcela_inicial;
    g_numero_parcelas = numero_parcelas;
    g_taxa_juros = taxa_juros;
}

double proximaParcela()
{   
    return g_parcela_atual *= (1 + g_taxa_juros/100);
}

int main(int argc, char** argv)
{    
    double S = 200;
    int N = 5;
    double J = 1;
 
    inicializarParametros(S, N, J);
    
    printf("Parcelas do financiamento: ");
    
    for (int i = 0; i < N; i++)
    {
        printf("%.2f ", g_parcela_atual);
        proximaParcela();
    }
}

Parcelas do financiamento: 200.00 202.00 204.02 206.06 208.12 

## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [38]:
#include <stdio.h>
#include <stdlib.h>

struct Emprestimo {
    double parcela_inicial;
    int num_parcelas;
    double taxa_juros;
    
    double parcela_atual;
    int num_parcela_atual;
    
    int completo;
};

struct Emprestimo* emprestimo_new(double parcela_inicial, int num_parcelas, double taxa_juros)
{
    struct Emprestimo* t = malloc(sizeof(*t));
    
    t->parcela_inicial = parcela_inicial;
    t->num_parcelas = num_parcelas;
    t->taxa_juros = taxa_juros;
    
    // começa em zero pra garantir que proximaParcela()
    // retorna a primeira parcela na primeira vez que
    // é chamado
    t->num_parcela_atual = 0;
    t->parcela_atual = 0;
    
    t->completo = 0;
    
    return t;
}

void emprestimo_delete(struct Emprestimo* t)
{
    free(t);
}

double emprestimo_proximaParcela(struct Emprestimo* t)
{
    if (t->num_parcela_atual == 0)
    {
        t->num_parcela_atual++;
        return t->parcela_atual = t->parcela_inicial;
    }
    
    else if (!t->completo)
    {
        t->num_parcela_atual++;
        t->parcela_atual *= (1 + t->taxa_juros/100);
    }
    
    if (t->num_parcela_atual >= t->num_parcelas)
    {
        t->completo = 1;
    }
    
    return t->parcela_atual;
}

int main(int argc, char** argv)
{
    double S = 200;
    int N = 5;
    double J = 1;
    
    struct Emprestimo* t = emprestimo_new(S, N, J);
    
    printf("Parcelas do financiamento: ");
    while(!t->completo)
    {
        printf("%.2f ", emprestimo_proximaParcela(t));
    }
    
    emprestimo_delete(t);
}

Parcelas do financiamento: 200.00 202.00 204.02 206.06 208.12 

## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [2]:
#include <stdio.h>
#include <stdlib.h>

struct Emprestimo {
    double parcela_inicial;
    int num_parcelas;
    double taxa_juros;
    
    double parcela_atual;
    int num_parcela_atual;
    
    int completo;
};

struct Emprestimo* emprestimo_new(double parcela_inicial, int num_parcelas, double taxa_juros)
{
    struct Emprestimo* t = malloc(sizeof(*t));
    
    t->parcela_inicial = parcela_inicial;
    t->num_parcelas = num_parcelas;
    t->taxa_juros = taxa_juros;
    
    // começa em zero pra garantir que proximaParcela()
    // retorna a primeira parcela na primeira vez que
    // é chamado
    t->num_parcela_atual = 0;
    t->parcela_atual = 0;
    
    t->completo = 0;
    
    return t;
}

void emprestimo_delete(struct Emprestimo* t)
{
    free(t);
}

// a mistura de underscore e camelCase é um abuso de notação
// pra indicar que é uma "função membro" da "classe" Emprestimo
// (e.g. Emprestimo.proximaParcela())
double emprestimo_proximaParcela(struct Emprestimo* t)
{
    if (t->num_parcela_atual == 0)
    {
        t->num_parcela_atual++;
        return t->parcela_atual = t->parcela_inicial;
    }
    
    else if (!t->completo)
    {
        t->num_parcela_atual++;
        t->parcela_atual *= (1 + t->taxa_juros/100);
    }
    
    if (t->num_parcela_atual >= t->num_parcelas)
    {
        t->completo = 1;
    }
    
    return t->parcela_atual;
}

int todosEmprestimosCompletos(struct Emprestimo** t, int size)
{
    int f = 1;
    
    for (int i = 0; i < size; i++)
    {
        if (!t[i]->completo)
        {
            f = 0;
            break;
        }
    }
    
    return f;
}

void deletarArrayEmprestimos(struct Emprestimo** t, int size)
{
    for (int i = 0; i < size; i++)
    {
        emprestimo_delete(t[i]);
    }
}

int main(int argc, char** argv)
{
    struct Emprestimo* t[2] = {emprestimo_new(200, 5, 1), emprestimo_new(500, 7, 2)};
    int t_size = 2;
    
    while(!todosEmprestimosCompletos(t, 2))
    {
        for(int i = 0; i < t_size; i++)
        {
            if (t[i]->completo)
            {
                continue;
            }
            
            emprestimo_proximaParcela(t[i]);
            printf("Emprestimo %d: parcela %d eh %.2f\n", i+1, t[i]->num_parcela_atual, t[i]->parcela_atual);
        }
    }

    deletarArrayEmprestimos(t, t_size);
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
